In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn 
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

Using TensorFlow backend.


### Data Augmentation(configuring)

In [4]:
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

img = load_img('D:/Projects/Apple Classifier/training_data/apples/images (0).jpg',target_size=(150,150))# this is the image to test the data augmentation
x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

# the .flow() command below generates batches of randomly transformed images
# and saves the results to the `preview/` directory
i = 0
for batch in datagen.flow(x, y = [1],batch_size=1,
                          save_to_dir='D:/Projects/Apple Classifier/preview', save_prefix='apple', save_format='jpeg'):
    i += 1
    if i > 20:
        break  

## First Model


In [6]:
from keras.models import Sequential    #seq model
from keras.layers import Conv2D, MaxPooling2D  #CNN
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (3, 3),padding = 'same', input_shape=(150, 150,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3),padding = 'same',activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3),padding = 'same',activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64,activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation = 'softmax'))

model.compile(loss='binary_crossentropy', optimizer='rmsprop',metrics=['accuracy'])

### Data Augmentation(Preparing data) 

In [8]:
batch_size = 10

#ugmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

#augmentation configuration we will use for validation:only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in training_data/apples and indefinitely generate batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        'D:/Projects/Apple Classifier/training_data', target_size=(150, 150), batch_size=batch_size, class_mode='binary')

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        'D:/Projects/Apple Classifier/validation_data', target_size=(150, 150), batch_size=batch_size, class_mode='binary')

Found 105 images belonging to 2 classes.
Found 30 images belonging to 2 classes.


Now, we can use these data generators while training our model, as train() takes data generators as input ,fit_generator

In [11]:
model.fit_generator(
        train_generator,
        steps_per_epoch = 4, #samples/batch_size
        epochs=50,
        validation_data=validation_generator,
        validation_steps=800 // batch_size)

#Saving the model
model.save_weights('D:/Projects/Apple Classifier/savedmodel/first_try.h5')

Epoch 1/50
4/4 [==============================] - 29s 7s/step - loss: 9.9640 - acc: 0.3750 - val_loss: 7.4331 - val_acc: 0.5338
Epoch 2/50
4/4 [==============================] - 25s 6s/step - loss: 9.5654 - acc: 0.4000 - val_loss: 7.4331 - val_acc: 0.5338
Epoch 3/50
4/4 [==============================] - 26s 6s/step - loss: 10.2942 - acc: 0.3543 - val_loss: 7.4730 - val_acc: 0.5313
Epoch 4/50
4/4 [==============================] - 25s 6s/step - loss: 7.6979 - acc: 0.5171 - val_loss: 7.3933 - val_acc: 0.5363
Epoch 5/50
4/4 [==============================] - 24s 6s/step - loss: 9.9640 - acc: 0.3750 - val_loss: 7.4531 - val_acc: 0.5325
Epoch 6/50
4/4 [==============================] - 24s 6s/step - loss: 10.7611 - acc: 0.3250 - val_loss: 7.4730 - val_acc: 0.5313
Epoch 7/50
4/4 [==============================] - 25s 6s/step - loss: 11.2052 - acc: 0.2971 - val_loss: 7.4531 - val_acc: 0.5325
Epoch 8/50
4/4 [==============================] - 25s 6s/step - loss: 9.1669 - acc: 0.4250 - val_loss

KeyboardInterrupt: 

In [42]:
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
import cv2, numpy as np

def VGGmodel(weights_path=None):
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(150,150,3)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
    model.add(Flatten())
    """ Top 3 layers would be removed.
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1000, activation='softmax'))"""

    if weights_path:
        model.load_weights(weights_path)

    return model

In [46]:
from keras import applications
batch_size = 1
datagen = ImageDataGenerator(rescale=1. / 255)

generator = datagen.flow_from_directory(
        'D:/Projects/Apple Classifier/training_data',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode=None,  
        shuffle=False) 

# build the VGG16 network
path = 'D:/Weights/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5';
model = VGGmodel(path)

print("Extracting the bottle-eck features for training data......")
bottleneck_features_train = model.predict_generator(generator, steps = len(generator))
print(bottleneck_features_train.shape)

# save the output as a Numpy array
np.save(open('D:/Projects/Apple Classifier/bottleneck_features_train.npy', 'wb'), bottleneck_features_train)

generator = datagen.flow_from_directory(
        'D:/Projects/Apple Classifier/validation_data',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
print("Extracting the bottle-eck features for validation data......")
bottleneck_features_validation = model.predict_generator(generator, steps = len(generator))
print(bottleneck_features_validation.shape)

#save the output as a Numpy array
np.save(open('D:/Projects/Apple Classifier/bottleneck_features_validation.npy', 'wb'), bottleneck_features_validation)

Found 105 images belonging to 2 classes.


C:\Users\shantanu\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  # Remove the CWD from sys.path while we load stuff.
C:\Users\shantanu\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  if sys.path[0] == '':
C:\Users\shantanu\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu")`
  app.launch_new_instance()
C:\Users\shantanu\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu")`
C:\Users\shantanu\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:22: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (3, 3), activa

Extracting the bottle-eck features for training data......
(105, 8192)
Found 30 images belonging to 2 classes.
Extracting the bottle-eck features for validation data......
(30, 8192)


In [68]:
train_data = np.load(open('D:/Projects/Apple Classifier/bottleneck_features_train.npy','rb'))
train_labels = np.array([0] * 65 + [1] * 40)

validation_data = np.load(open('D:/Projects/Apple Classifier/bottleneck_features_validation.npy','rb'))
validation_labels = np.array([0] * 14 + [1] * 16)
train_data = np.reshape(train_data, train_data.shape + (1,));
validation_data = np.reshape(validation_data,  validation_data.shape + (1,));
print(train_data.shape, validation_data.shape, validation_labels.shape)

model = Sequential()
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='softmax'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(train_data, train_labels,
          epochs=50,
          batch_size=10,
          validation_data=(validation_data, validation_labels))
model.save_weights('D:/Projects/Apple Classifier/savedmodel/bottleneck_fc_model.h5')

(105, 8192, 1) (30, 8192, 1) (30,)
Train on 105 samples, validate on 30 samples
Epoch 1/50
105/105 [==============================] - 3s 28ms/step - loss: 9.8691 - acc: 0.3810 - val_loss: 7.4398 - val_acc: 0.5333
Epoch 2/50
105/105 [==============================] - 1s 9ms/step - loss: 9.8691 - acc: 0.3810 - val_loss: 7.4398 - val_acc: 0.5333
Epoch 3/50
105/105 [==============================] - 1s 9ms/step - loss: 9.8691 - acc: 0.3810 - val_loss: 7.4398 - val_acc: 0.5333
Epoch 4/50
105/105 [==============================] - 1s 9ms/step - loss: 9.8691 - acc: 0.3810 - val_loss: 7.4398 - val_acc: 0.5333
Epoch 5/50
105/105 [==============================] - 1s 9ms/step - loss: 9.8691 - acc: 0.3810 - val_loss: 7.4398 - val_acc: 0.5333
Epoch 6/50
105/105 [==============================] - 1s 9ms/step - loss: 9.8691 - acc: 0.3810 - val_loss: 7.4398 - val_acc: 0.5333
Epoch 7/50
105/105 [==============================] - 1s 9ms/step - loss: 9.8691 - acc: 0.3810 - val_loss: 7.4398 - val_acc: 0.

KeyboardInterrupt: 